In [1]:
import pandas as pd
import os
import csv
import numpy as np
from matplotlib import pyplot as plt
%matplotlib

Using matplotlib backend: Qt5Agg


In [ ]:
import platform
print(platform.python_version())

In [ ]:
from scipy.interpolate import UnivariateSpline
from scipy.optimize import curve_fit
from scipy import interpolate, optimize

In [ ]:
#lectura de archivos
file_endings = ['.csv', ' - dinamica.csv', ' - perfil.csv', ' - flir.csv']

#Experimento es una lista que guarda todos los experimentos y accede a los DF que son los definidos en file_endings
class Experimento(object):
    def __init__(self, filename):
        self.files = ['{}{}'.format(filename, file_ending) for file_ending in file_endings]
            
        pandas_settings = {
            'delimiter': ';',
            'thousands': '.',
            'decimal': ',',
            'engine': 'python',
        }
        #se definen los 4 DF param, dinamica, perfil y flir.
        self.param = pd.read_csv(self.files[0], **pandas_settings)
        self.dinamica = pd.read_csv(self.files[1], **pandas_settings)
        self.perfil = pd.read_csv(self.files[2], **pandas_settings)
    #    self.flir = pd.read_csv(self.files[3], **pandas_settings)
        
        #paso los resultados de dinamica a param para poder usarlos facilmente
        self.param['result: largo final'] = self.dinamica['avance: distancia desde punto eyeccion'].iloc[-1]
        self.param['result: largo total'] = self.dinamica['avance: largo total flujo'].iloc[-1]
        self.param['result: ancho max final'] = self.dinamica['avance: ancho maximo'].iloc[-1]
        self.param['result: espesor max final'] = self.perfil['perfil: espesor'].max()
        self.param['result: distancia espesor final'] = self.perfil['perfil: largo'].iloc[self.perfil['perfil: espesor'].argmax()]
        
    #se guarda el nuevo archivo con los datos sacados de distintos DF  
    def to_csvs(self, safe=False):
        if safe:
            try:
                for file in self.files:
                    os.rename(file, '{}.old'.format(file))
            except Exception as e:
                print('The files already exists', e)
                return
        else:
            for file in self.files:
                os.remove(file)
        
        self.param.to_csv(self.files[0], sep=';')
        self.dinamica.to_csv(self.files[1], sep=';')
        self.perfil.to_csv(self.files[2], sep=';')
 #       self.flir.to_csv(self.files[3], sep=';')
    #a ver si ya se ha hecho este proceso con los archivos y generar una version .old    
    def undone(self):
        for file in self.files:
            os.rename('{}.old'.format(file), file)
    
    #funcion para generar rectas segun x datos de DF dinamica
    def fit_to_two_curves(self):
        def two_lines(x, a, b, c, d):
            one = a*x + b
            two = c*x + d
            return np.maximum(one, two)
        
        x = np.array(self.dinamica['avance: tiempo'])
        y = np.array(self.dinamica['avance: largo total'])
    
    #no entiendo pa que wea sirve esta funcion...claramente es un plot de desglose, pero pa que? esos datos meh.
    def plot_desglose(self, param1, param2):
        fig = plt.figure()
        ax = fig.add_subplot(1,1,1)

        ax.plot(self.flir['FLIR: rango 3'], self.flir['FLIR: rango 4'])
        fig.show()
        
    def __repr__(self):
        return self.files[0]
        
    def __str__(self):
        return self.files[0]

In [66]:
#lista con los nombres de los experimentos
nameList = [
        '5cc50C',
        '1cc54C',
        '25cc54C',
        '10cc60C',
        '25cc60C',
        '1cc59C',
        '1cc47C',
        '1cc55C',
        '25cc58C',
        '25cc70C',
        '10cc67C',
        '10cc59C',
        '5cc63C',
        '5cc59C',
        '10cc49C',
        '25cc62C',
        '1cc62C',
        '5cc53C',
        '5a25cc56C',
        '5cc57C',
        '25cc50C',
        '10cc55C',
        '1cc24C',
        '10cc57C',
        '10cc24C',
        '5cc55C',
        '5cc58C',
        '1cc48C',
        '1cc51C',
        '5cc48C',
        '10cc50C',
        '1cc54,7C',
        '1cc52C',

]

In [67]:
Experimentos = [Experimento(filename) for filename in nameList]

In [69]:
Experimentos[2]

5cc58C.csv

In [70]:
exp1 = Experimentos[0]
exp2 = Experimentos[1]
exp3 = Experimentos[2]
exp4 = Experimentos[3]

In [71]:
Experimentos[0].param.head() #imprime las primeras 5 filas del exp 0 

,Unnamed: 0,Nombre exp,fecha,exp #,uso manjar #,caudal,temperatura maxima,temperatura ambiente,angulo inclinacion,densidad manjar (t ambiente),...,result: largo final desde punto eyeccion,result: largo total,result: ancho maximo final,result: tiempo total medicion,result: espesor maximo,result: distancia a espesor maximo,result: largo final,result: ancho max final,result: espesor max final,result: distancia espesor final
0,0,1cc51C,29/09/2016,29,2,0.000001,51,24.7,10.4,1240,...,NaN,0.398,NaN,54208.68333,0.019,0.2,0.33,0.285,0.019,0.2


In [72]:
Experimentos[0].dinamica.head()

,Unnamed: 0,avance: tiempo,avance: distancia desde punto eyeccion,avance: distancia hacia atrás,avance: largo total flujo,avance: ancho maximo
0,0,0.000000,0.00,0.000,0.000,0.000
1,1,24.350000,0.01,0.015,0.025,0.050
2,2,38.516667,0.02,0.020,0.040,0.060
3,3,53.683333,0.03,0.025,0.055,0.070
4,4,74.850000,0.04,0.027,0.067,0.082


In [73]:
#lista para armar graficos de perfil agrupados por caudal
listapalplot = []
#no entiendo como la lista guarda las cosas del diccionario...
#es una lista donde en cada value hay un diccionario por experimento
#ANOTAR EN UNA HOJA COMO SE VE Y USA ESTO PLS
for exp in Experimentos:
    holder = {} #es un diccionario 
    holder['nombre'] = exp.param['Nombre exp'][0]#solo el valor de la primera fila
    holder['espesor'] = exp.perfil['perfil: espesor']#toda la columna
    holder['largo'] = exp.perfil['perfil: largo']#toda la columna
    holder['caudal'] = exp.param['caudal '][0]#solo el valor de la primera fila
    listapalplot.append(holder)

In [74]:
from itertools import groupby

count = 0

#se arman grupos segun el valor del caudal
#key es la lista donde estan los diccionarios de exp y DF
#group es que se agrupa segun el valor de 'caudal' en este caso
for key, group in groupby(listapalplot, key=lambda x: x['caudal']):
    print(key)
    count += 1
    for exp in group:
        #muestra el nombre del grupo y los elementos que tiene
        print(exp['nombre']) 

1e-06
1cc51C
1cc55C
5e-06
5cc58C
1e-05
10cc50C


In [75]:
count

3

In [76]:
#GRAFICO 1:
#empezamo el ploteo de todos los perfiles finales agrupados segun caudal
fig = plt.figure()
axes = []

#este plot hace cuantos subplot necesite segun un contador que da el numero de
#grupos generados en el groupby
for x in range(count):
    axes.append(fig.add_subplot(count, 1, x + 1))

#GRAFICO 1: UNA COLUMNA DE GRAFICOS CON VARIAS FILAS SEGUN GRUPO
#se generan los ejes de cada plot
#i es el indice de lista y luego cada value en lista (???)
for i, (key, group) in enumerate(groupby(listapalplot, key=lambda x: x['caudal'])):
    print(i, key)
    for exp in group:
        axes[i].plot(exp['largo'], exp['espesor'])
        axes[i].legend([key])

In [78]:
#GRAFICO 2: X COLUMNA DE GRAFICOS CON Y FILAS SEGUN GRUPO..se define en los parentesis de ax
fig = plt.figure()
ax = fig.add_subplot(3,2,1)
ax2 = fig.add_subplot(3,2,2)
ax3 = fig.add_subplot(3,2,3)

for x in Experimentos:
    ax.plot(x.perfil['perfil: largo'], x.perfil['perfil: espesor'], 'o')
    ax2.plot(x.perfil['perfil: espesor'], x.perfil['perfil: largo'], 'x')
    ax3.plot(x.perfil['perfil: espesor'], x.perfil['perfil: largo'], 'x')

fig.show()

In [79]:
#lista para armar figures de velocidad segun Q
listadinamica = []
#es una lista donde en cada value hay un diccionario por experimento
#ANOTAR EN UNA HOJA COMO SE VE Y USA ESTO PLS
for exp in Experimentos:
    holder = {} #es un diccionario 
    holder['nombre'] = exp.param['Nombre exp'][0]#solo el valor de la primera fila
    holder['tiempo'] = exp.dinamica['avance: tiempo']#toda la columna
    holder['distancia'] = exp.dinamica['avance: distancia desde punto eyeccion']#toda la columna
    holder['caudal'] = exp.param['caudal '][0]#solo el valor de la primera fila
    listadinamica.append(holder)

In [84]:
#empezamo el ploteo de todas las dinamicas agrupados segun caudal
fig = plt.figure()
axes = []

#este plot hace cuantos subplot necesite segun un contador que da el numero de
#grupos generados en el groupby
for x in range(count):
    axes.append(fig.add_subplot(count, 1, x + 1))

#UNA COLUMNA DE GRAFICOS CON VARIAS FILAS SEGUN GRUPO
#se generan los ejes de cada plot
#i es el indice de lista y luego cada value en lista (???)
for i, (key, group) in enumerate(groupby(listadinamica, key=lambda x: x['caudal'])):
    print(i, key)
    for exp in group:
        axes[i].loglog(exp['tiempo'], exp['distancia'])
        axes[i].legend([key])

0 1e-06
1 5e-06
2 1e-05


In [86]:
Experimentos[0].dinamica.head()

,Unnamed: 0,avance: tiempo,avance: distancia desde punto eyeccion,avance: distancia hacia atrás,avance: largo total flujo,avance: ancho maximo
0,0,0.000000,0.00,0.000,0.000,0.000
1,1,24.350000,0.01,0.015,0.025,0.050
2,2,38.516667,0.02,0.020,0.040,0.060
3,3,53.683333,0.03,0.025,0.055,0.070
4,4,74.850000,0.04,0.027,0.067,0.082
